## Finding the closest restaurants to open spaces, food trucks, farmers markets, and murals

In [1]:
import pandas as pd
import requests
import json
import numpy as np
import time as tm
import geopy.distance

In [21]:
# returns number of stations requested by 'k'
# there's a column called "NUM STATIONS" for each open space that gives the number of stations 
# that are actually within the distance 'd' 

In [40]:
# constants
k = 10 # k-nearest neighbors
d = 0.3 # maximum distance in km

In [23]:
# file locations
open_space_source = 'data/raw/Open_Space.geojson'
restaurants_source = 'restaurants.csv'
stations_source = 'data/raw/current_bluebikes_stations.csv'
food_truck_source = 'data/raw/food_truck_schedule.geojson'
farmers_market_source = 'data/raw/winter_farmers_markets.csv'
murals_source = 'data/raw/murals.csv'

In [24]:
restaurants = pd.read_csv(restaurants_source)

In [25]:
restaurants

,businessname,dbaname,address,city,state,zip,licstatus,licensecat,descript,license_add_dt_tm,dayphn_cleaned,property_id,latitude,longitude
0,311,NaN,605 Tremont,Roxbury,MA,2118,Active,FS,Eating & Drinking,2022-08-01 15:00:41.393+00,NaN,137766.0,42.343140,-71.073641
1,# 7 RESTAURANT,NaN,225 Grove,West Roxbury,MA,2132,Active,FT,Eating & Drinking w/ Take Out,2007-03-23 19:20:59+00,0.000000e+00,156965.0,42.261873,-71.157411
2,100 Federal Market & Barista- 10th Floor,NaN,100 Federal,Boston,MA,2110,Active,FS,Eating & Drinking,2023-08-09 16:00:26.99+00,6.174345e+09,57150.0,42.354770,-71.056130
3,100 Federal Market & Barista- 11th Floor,NaN,100 Federal,Boston,MA,2110,Active,FT,Eating & Drinking w/ Take Out,2021-01-12 15:37:24.463+00,6.174345e+09,57150.0,42.354770,-71.056130
4,110 Grill,NaN,25 Jan Karski,Dorchester,MA,2125,Active,FS,Eating & Drinking,2018-11-29 19:14:04.217+00,9.786929e+09,421286.0,42.325375,-71.061553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066,Ziggy's,NaN,583 Washington,Brighton,MA,2135,Active,FT,Eating & Drinking w/ Take Out,2020-10-23 16:17:07.17+00,6.178334e+09,351055.0,42.350379,-71.165212
3067,Zo,NaN,92 State,Boston,MA,2109,Active,FT,Eating & Drinking w/ Take Out,2011-12-16 14:44:46+00,6.179016e+09,170003.0,42.359222,-71.055324
3068,Zo Greek,NaN,225 Franklin,Boston,MA,2110,Active,FS,Eating & Drinking,2024-01-19 19:52:11.823+00,8.574496e+09,158047.0,42.356135,-71.054069
3069,Zuma Restaurant,NaN,1 Dalton,Boston,MA,2115,Active,FS,Eating & Drinking,2018-02-16 15:18:58.307+00,8.574492e+09,351070.0,42.345753,-71.084012


In [26]:
# getting average coordinate of each open space using max/min long/lat
def average_coord (i):
    num_coords = len(data['features'][i]['geometry']["coordinates"][0][0])
    min_lat = 1000
    max_lat = -1000
    min_long = 1000
    max_long = -1000
    for j in range(0, num_coords):
        lat = data['features'][i]['geometry']["coordinates"][0][0][j][0]
        long = data['features'][i]['geometry']["coordinates"][0][0][j][1]
        if lat < min_lat: 
            min_lat = lat
        if lat > max_lat:
            max_lat = lat
        if long < min_long: 
            min_long = long
        if long > max_long:
            max_long = long
    return [(min_lat+max_lat)/2, (min_long+max_long)/2]

## Open spaces

In [27]:
f = open(open_space_source)
data = json.load(f)

In [28]:
open_spaces = pd.DataFrame(columns=['OBJECTID', 'OS_ID', 'PARK_ID', 'Name', 'Avg. Lat', 'Avg. Long'])


In [29]:
for i in range(0, len(data['features'])):
    oid = data['features'][i]['properties']['OBJECTID']
    osid = data['features'][i]['properties']['OS_ID']
    parkid = data['features'][i]['properties']['PARK_ID']
    name = data['features'][i]['properties']['SITE_NAME']
    ll = average_coord(i)
    open_spaces.loc[len(open_spaces.index)] = [oid, osid, parkid, name, ll[1], ll[0]]  


In [16]:
open_spaces

,OBJECTID,OS_ID,PARK_ID,Name,Avg. Lat,Avg. Long
0,1,2000,None,Wilson Park,42.340991,-71.145929
1,2,156,156,James H. Roberts Playground,42.359526,-71.125332
2,3,284,284,Smith Playground,42.365072,-71.131046
3,4,115,115,Fern Square,42.359534,-71.133291
4,5,259,259,Ringer Playground,42.350456,-71.138129
...,...,...,...,...,...,...
529,530,7273,7273,Mahoney Park,42.342008,-71.051382
530,531,2106,2106,Boundary I,42.269348,-71.137990
531,1340,7275,7275,Sprague Pond,42.233786,-71.138006
532,2541,7276,7276,Codman Square,42.290418,-71.071082


In [17]:
open_spaces.to_csv('open_space_coords.csv', index=False)

In [30]:
meta_dists = []

for i in range(len(open_spaces)):
    space = (open_spaces["Avg. Lat"][i], open_spaces["Avg. Long"][i])
    dists = []

    for j in range(len(restaurants)):
        stat = (restaurants['latitude'][j], restaurants['longitude'][j])
        distance = geopy.distance.geodesic(space, stat).km
        dists += [(distance, restaurants['businessname'][j])]

    meta_dists += [sorted(dists)[0:k]]

In [31]:
meta_dists

[[(0.05532462131540542, 'DOLCE VITA (1696 Comm. Ave.)'),
  (0.057532698888595246, 'Land of Fire Pizzeria'),
  (0.07586323997700861, 'Vaisakhi Indian Kitchen'),
  (0.08010919069293913, 'Andalus Pizza & Cafe'),
  (0.3650619012955277, 'Bamboo Thai Restaurant'),
  (0.37463040382651475, 'TASCA RESTAURANT'),
  (0.38500829741453774, 'La Fonda Boston'),
  (0.385896332022384, 'Milano Pizzeria and Pasta'),
  (0.40896150270811793, 'BRIGHTON CAFE'),
  (0.4147594728435499, 'NEW YORK FRIED CHICKEN & PIZZA')],
 [(0.16333574635418635, 'Dunkin Donuts (219 Cambridge)'),
  (0.39822632768369315, 'SEC Cafe / Kitchen'),
  (0.46725380877230427, 'Swiss Bakers'),
  (0.5580272332921417, 'Dunkin Donuts (209 North Harvard St.)'),
  (0.5651195063038362, 'THE FIELD HOUSE'),
  (0.5704302949772433, 'At Union'),
  (0.577702700880576, "Sloane's"),
  (0.5802570606298147, 'Double Tree Guest Suites'),
  (0.6151003874815801, 'SPANGLER STUDENT CENTER'),
  (0.6161210801544588, 'Starbucks Coffee')],
 [(0.1929246352747221, 'WB

In [32]:
meta_dists_saved = meta_dists

In [34]:
open_spaces_top_restaurants = pd.DataFrame(columns=['OBJECTID', 'OS_ID', 'PARK_ID', 'Name', 'Avg. Lat', 'Avg. Long', 'NUM STATIONS'])


for i in range(k):
    open_spaces_top_restaurants[("Station {}".format(str(i+1)))] = []
    open_spaces_top_restaurants[("Station {} Dist ".format(str(i+1)))] = []

In [35]:
for i in range(len(meta_dists)):
    
    
    
    oid = open_spaces['OBJECTID'][i]
    osid = open_spaces['OS_ID'][i]
    parkid = open_spaces['PARK_ID'][i]
    name = open_spaces['Name'][i]
    lat = open_spaces['Avg. Lat'][i]
    long = open_spaces['Avg. Long'][i]
    
    
    to_return = [oid, osid, parkid, name, lat, long] 
    to_return_2 = []
    
    num = 0
    
    for j in range(k):
        stat = meta_dists[i][j][1]
        stat_d = meta_dists[i][j][0]
        
        if stat_d < d:
            num += 1
        
        to_return_2 += [stat, stat_d]
        
    
    to_return_2 = [num] + to_return_2

    to_return += to_return_2
    
    open_spaces_top_restaurants.loc[len(open_spaces_top_restaurants.index)] = to_return


In [37]:
open_spaces_top_restaurants

,OBJECTID,OS_ID,PARK_ID,Name,Avg. Lat,Avg. Long,NUM STATIONS,Station 1,Station 1 Dist,Station 2,...,Station 6,Station 6 Dist,Station 7,Station 7 Dist,Station 8,Station 8 Dist,Station 9,Station 9 Dist,Station 10,Station 10 Dist
0,1,2000,None,Wilson Park,42.340991,-71.145929,4,DOLCE VITA (1696 Comm. Ave.),0.055325,Land of Fire Pizzeria,...,TASCA RESTAURANT,0.374630,La Fonda Boston,0.385008,Milano Pizzeria and Pasta,0.385896,BRIGHTON CAFE,0.408962,NEW YORK FRIED CHICKEN & PIZZA,0.414759
1,2,156,156,James H. Roberts Playground,42.359526,-71.125332,1,Dunkin Donuts (219 Cambridge),0.163336,SEC Cafe / Kitchen,...,At Union,0.570430,Sloane's,0.577703,Double Tree Guest Suites,0.580257,SPANGLER STUDENT CENTER,0.615100,Starbucks Coffee,0.616121
2,3,284,284,Smith Playground,42.365072,-71.131046,5,WBZ Cafe,0.192925,Starbucks Coffee,...,Swiss Bakers,0.310013,SEC Cafe / Kitchen,0.449895,Spring Shabu-Shabu,0.484188,Bright-Landry Arena Briggs Cafe,0.500444,Lavietes Pavillion,0.500444
3,4,115,115,Fern Square,42.359534,-71.133291,0,Star Market #4572,0.389477,Dunkin Donuts (209 North Harvard St.),...,Starbucks Coffee,0.469652,O'Brien's Pub,0.475680,9 Thai Express,0.494133,LULUS' ALLSTON,0.494133,Ten Second Noodles,0.494133
4,5,259,259,Ringer Playground,42.350456,-71.138129,5,Harry's Bar & Grill,0.187505,Jongro BBQ,...,Charlie's Pizza & Cafe,0.302872,DUNKIN DONUTS (ALLSTON),0.330346,TAQUERIA EL CARRIZAL,0.333330,TWIN DONUTS,0.340521,KFC,0.354108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529,530,7273,7273,Mahoney Park,42.342008,-71.051382,10,Vester,0.194354,Bar Volpe,...,Lenox Sophia,0.267007,SHEA'S TAVERN,0.268967,Boardwalk Southie,0.286002,Broadway Pastry,0.287506,Pizzeria 260,0.288607
530,531,2106,2106,Boundary I,42.269348,-71.137990,0,Domino's,0.625325,DUNKIN (STATION DONUTS),...,Pleasant Cafe,1.051602,Golden House,1.423225,St. George Orthodox Church,1.425811,D Coal Pot,1.461365,Tori Japan,1.462568
531,1340,7275,7275,Sprague Pond,42.233786,-71.138006,0,Mt. Olympic Pizza,0.601379,DUNKIN DONUTS/HYDE PK AVE,...,CAVAN CAFE,2.128008,ONLY ONE JAMAICAN RESTAURANT,2.389158,Mcdonalds,2.486498,CAPPY'S PIZZA,2.622616,Subway,2.633464
532,2541,7276,7276,Codman Square,42.290418,-71.071082,5,McDonald's,0.065110,ORIENTAL HOUSE,...,Essential Herb & Juice Cafe,0.418721,J. P. Roti Shop,0.477293,Raphael's Restaurant,0.477746,Kentucky Fried Chicken,0.529568,Daily Table,0.608913


In [38]:
open_spaces_top_stations.to_csv('open_spaces_nearest_restaurants_k_{}_d_{}.csv'.format(str(k), str(d)), index=False)

## Food trucks

In [41]:
f = open(food_truck_source)
data = json.load(f)

In [42]:
food_trucks = pd.DataFrame(columns=['OBJECTID', 'Name', 'Lat', 'Long'])

In [43]:
data['features'][0]['properties']['ObjectId']

1

In [44]:
for i in range(len(data['features'])):
    oid = data['features'][i]['properties']['ObjectId']
    name = data['features'][i]['properties']['Truck']
    lat = data['features'][i]['properties']['y']
    long = data['features'][i]['properties']['x']
    food_trucks.loc[len(food_trucks.index)] = [oid, name, lat, long]

In [45]:
food_trucks

,OBJECTID,Name,Lat,Long
0,1,Indian Street Cravings,42.350534,-71.075124
1,2,Moyzilla,42.350534,-71.075124
2,3,Hungry Nomads,42.350534,-71.075124
3,4,Chicken and Rice Guys,42.350534,-71.075124
4,5,Tacos Calleteco,42.350534,-71.075124
5,6,Hungry Nomads,42.350534,-71.075124
6,7,Indian Street Cravings,42.360581,-71.057741
7,8,Chick-fil-A,42.377834,-71.051037
8,9,Chick-fil-A,42.377834,-71.051037
9,10,Indian Street Cravings,42.340478,-71.088974


In [48]:
meta_dists = []

for i in range(len(food_trucks)):
    truck = (food_trucks["Lat"][i], food_trucks["Long"][i])
    dists = []

    for j in range(len(restaurants)):
        stat = (restaurants['latitude'][j], restaurants['longitude'][j])
        distance = geopy.distance.geodesic(truck, stat).km
        dists += [(distance, restaurants['businessname'][j])]

    meta_dists += [sorted(dists)[0:k]]

In [49]:
food_trucks_top_restaurants = pd.DataFrame(columns=['OBJECTID', 'Name', 'Avg. Lat', 'Avg. Long', 'NUM STATIONS'])


for i in range(k):
    food_trucks_top_restaurants[("Station {}".format(str(i+1)))] = []
    food_trucks_top_restaurants[("Station {} Dist ".format(str(i+1)))] = []

In [50]:

for i in range(len(meta_dists)):
    
    oid = food_trucks['OBJECTID'][i]
    name = food_trucks['Name'][i]
    lat = food_trucks['Lat'][i]
    long = food_trucks['Long'][i]  
    
    
    
    to_return = [oid, name, lat, long] 
    to_return_2 = []
    
    num = 0
    
    for j in range(k):
        stat = meta_dists[i][j][1]
        stat_d = meta_dists[i][j][0]
        
        if stat_d < d:
            num += 1
        
        to_return_2 += [stat, stat_d]
        
    
    to_return_2 = [num] + to_return_2

    to_return += to_return_2
    
    food_trucks_top_restaurants.loc[len(food_trucks_top_restaurants.index)] = to_return


In [51]:
food_trucks_top_restaurants.to_csv('food_trucks_nearest_restaurants_k_{}_d_{}.csv'.format(str(k), str(d)), index=False)

In [52]:
food_trucks_top_restaurants

,OBJECTID,Name,Avg. Lat,Avg. Long,NUM STATIONS,Station 1,Station 1 Dist,Station 2,Station 2 Dist,Station 3,...,Station 6,Station 6 Dist,Station 7,Station 7 Dist,Station 8,Station 8 Dist,Station 9,Station 9 Dist,Station 10,Station 10 Dist
0,1,Indian Street Cravings,42.350534,-71.075124,10,FINAGLE-A-BAGEL,0.053606,Smashburger #1725,0.064308,Yamato II,...,Insomnia Cookies,0.086964,Dig Inn,0.088554,Flik @ Skadden Arps Slate,0.100811,Chick-Fil-A,0.109922,THAI BASIL,0.135205
1,2,Moyzilla,42.350534,-71.075124,10,FINAGLE-A-BAGEL,0.053606,Smashburger #1725,0.064308,Yamato II,...,Insomnia Cookies,0.086964,Dig Inn,0.088554,Flik @ Skadden Arps Slate,0.100811,Chick-Fil-A,0.109922,THAI BASIL,0.135205
2,3,Hungry Nomads,42.350534,-71.075124,10,FINAGLE-A-BAGEL,0.053606,Smashburger #1725,0.064308,Yamato II,...,Insomnia Cookies,0.086964,Dig Inn,0.088554,Flik @ Skadden Arps Slate,0.100811,Chick-Fil-A,0.109922,THAI BASIL,0.135205
3,4,Chicken and Rice Guys,42.350534,-71.075124,10,FINAGLE-A-BAGEL,0.053606,Smashburger #1725,0.064308,Yamato II,...,Insomnia Cookies,0.086964,Dig Inn,0.088554,Flik @ Skadden Arps Slate,0.100811,Chick-Fil-A,0.109922,THAI BASIL,0.135205
4,5,Tacos Calleteco,42.350534,-71.075124,10,FINAGLE-A-BAGEL,0.053606,Smashburger #1725,0.064308,Yamato II,...,Insomnia Cookies,0.086964,Dig Inn,0.088554,Flik @ Skadden Arps Slate,0.100811,Chick-Fil-A,0.109922,THAI BASIL,0.135205
5,6,Hungry Nomads,42.350534,-71.075124,10,FINAGLE-A-BAGEL,0.053606,Smashburger #1725,0.064308,Yamato II,...,Insomnia Cookies,0.086964,Dig Inn,0.088554,Flik @ Skadden Arps Slate,0.100811,Chick-Fil-A,0.109922,THAI BASIL,0.135205
6,7,Indian Street Cravings,42.360581,-71.057741,10,Recreo Coffee,0.053291,Zaz Restaurant & Catering Corp.,0.053291,Alma Cantina,...,Loyal Nine,0.095547,HENNESSY'S OF BOSTON,0.096054,Saus Boston,0.102650,UNION OYSTER HOUSE,0.104292,DUNKIN DONUTS',0.115529
7,8,Chick-fil-A,42.377834,-71.051037,0,Dunkin Donuts,0.346651,Dovetail,0.402684,The Anchor,...,Style Cafe,0.520167,Pier 6 Restaurant,0.523641,Chattermark Distillers,0.598486,The Corner Restaurant,0.812823,AFC Sushi @ Shaws #1208,0.897188
8,9,Chick-fil-A,42.377834,-71.051037,0,Dunkin Donuts,0.346651,Dovetail,0.402684,The Anchor,...,Style Cafe,0.520167,Pier 6 Restaurant,0.523641,Chattermark Distillers,0.598486,The Corner Restaurant,0.812823,AFC Sushi @ Shaws #1208,0.897188
9,10,Indian Street Cravings,42.340478,-71.088974,10,Northeastern Univ/Curry Center,0.048330,Mamacita,0.081262,Wings Over Boston,...,N.U. Stetson Hall West,0.121011,Amelia's Taqueria,0.134413,Poke Bar & Kitchen,0.134651,Gyroscope/Sprout/Bos Hse of Pizza,0.161281,Chartwells Churchill Cafe,0.189075


## Farmers markets (winter only)

In [53]:
data = pd.read_csv(farmers_market_source)
data

,Market Name,Type,Address,Zip code,Latitude,Longitude,Neighborhood,Market days,Market Hours,First day of Market,Last day of Market,WIC and Senior Farmers Market Nutrition Program Vouchers,Accepts City of Boston Coupons
0,Brighton Farmers Market,In-Person,"640 Washington St, Brighton MA 02135",2135,42.347872,-71.153686,Brighton,"Every other Saturday: 01/27, 02/10, 02/24, 03/...",11am-2pm,01/27/24,03/30/24,WIC,NaN
1,Dorchester Winter Farmers Market,In-Person,"6 Norfolk St., Dorchester MA, 02124",2124,42.289960,-71.071840,Dorchester,Saturday,11am-2pm,01/27/23,03/30/24,"WIC, SFMNPV",NaN
2,East Boston Farmers Market,In-Person,"250 Sumner St, East Boston MA 02128",2128,42.375231,-71.040673,East Boston,Thursday,2pm-6pm,11/16/23,06/20/24,"WIC, SFMNPV",NaN
3,WeGrowMicrogreens,In-Person,"21 Norton St, Hyde Park MA 02136",2136,42.242854,-71.136693,Hyde Park,Friday,1pm-4:30pm,10/27/23,03/08/24,"WIC, SFMNPV",NaN
4,JP Centre St Farmers Market,In-Person,677 Centre St. Bank of America Parking Lot,2130,42.312173,-71.114370,Jamaica Plain,Saturday,12pm-3pm,11/11/24,03/30/24,"WIC, SFMNPV",NaN
5,Roslindale Farmers Market,In-Person,"Adams Park, Roslindale 02131",2131,42.286545,-71.128441,Roslindale,Sunday,11am-2pm,01/7/24,03/31/24,Neither,NaN
6,Dudley Town Common Farmers Market,In-Person,"11 Brook Ave, Roxbury MA 02119",2119,42.325115,-71.074876,Roxbury,Thursday,2:30pm-6:00pm,11/3/23,02/01/24,"WIC, SFMNPV",NaN


In [55]:
meta_dists = []

for i in range(len(data)):
    market = (data["Latitude"][i], data["Longitude"][i])
    dists = []

    for j in range(len(restaurants)):
        stat = (restaurants['latitude'][j], restaurants['longitude'][j])
        distance = geopy.distance.geodesic(truck, stat).km
        dists += [(distance, restaurants['businessname'][j])]

    meta_dists += [sorted(dists)[0:k]]

In [56]:
farmers_markets_top_restaurants = pd.DataFrame(columns=['OBJECTID', 'Name', 'Avg. Lat', 'Avg. Long', 'NUM STATIONS'])


for i in range(k):
    farmers_markets_top_restaurants[("Station {}".format(str(i+1)))] = []
    farmers_markets_top_restaurants[("Station {} Dist ".format(str(i+1)))] = []

In [57]:
for i in range(len(meta_dists)):
    
    name = data['Market Name'][i]
    lat = data['Latitude'][i]
    long = data['Longitude'][i]
    
    
    to_return = [i, name, lat, long] 
    to_return_2 = []
    
    num = 0
    
    for j in range(k):
        stat = meta_dists[i][j][1]
        stat_d = meta_dists[i][j][0]
        
        if stat_d < d:
            num += 1
        
        to_return_2 += [stat, stat_d]
        
    
    to_return_2 = [num] + to_return_2

    to_return += to_return_2
    
    farmers_markets_top_restaurants.loc[len(farmers_markets_top_restaurants.index)] = to_return


In [58]:
farmers_markets_top_restaurants

,OBJECTID,Name,Avg. Lat,Avg. Long,NUM STATIONS,Station 1,Station 1 Dist,Station 2,Station 2 Dist,Station 3,...,Station 6,Station 6 Dist,Station 7,Station 7 Dist,Station 8,Station 8 Dist,Station 9,Station 9 Dist,Station 10,Station 10 Dist
0,0,Brighton Farmers Market,42.347872,-71.153686,10,Mass Gen. Hosp./Tea Leaves & Coffee Beans,0.100539,Mass General Hospital Cafe,0.100539,BIG DADDY HOT DOGS,...,Massachusetts Eye & Ear Infirmary Eye Opener Cafe,0.213857,RIVERSIDE CAFE-LEV1,0.261843,"YAWKEY CENTER ""COFFEE SOUTH""",0.261843,Scampo,0.275161,Alibi,0.278161
1,1,Dorchester Winter Farmers Market,42.289960,-71.071840,10,Mass Gen. Hosp./Tea Leaves & Coffee Beans,0.100539,Mass General Hospital Cafe,0.100539,BIG DADDY HOT DOGS,...,Massachusetts Eye & Ear Infirmary Eye Opener Cafe,0.213857,RIVERSIDE CAFE-LEV1,0.261843,"YAWKEY CENTER ""COFFEE SOUTH""",0.261843,Scampo,0.275161,Alibi,0.278161
2,2,East Boston Farmers Market,42.375231,-71.040673,10,Mass Gen. Hosp./Tea Leaves & Coffee Beans,0.100539,Mass General Hospital Cafe,0.100539,BIG DADDY HOT DOGS,...,Massachusetts Eye & Ear Infirmary Eye Opener Cafe,0.213857,RIVERSIDE CAFE-LEV1,0.261843,"YAWKEY CENTER ""COFFEE SOUTH""",0.261843,Scampo,0.275161,Alibi,0.278161
3,3,WeGrowMicrogreens,42.242854,-71.136693,10,Mass Gen. Hosp./Tea Leaves & Coffee Beans,0.100539,Mass General Hospital Cafe,0.100539,BIG DADDY HOT DOGS,...,Massachusetts Eye & Ear Infirmary Eye Opener Cafe,0.213857,RIVERSIDE CAFE-LEV1,0.261843,"YAWKEY CENTER ""COFFEE SOUTH""",0.261843,Scampo,0.275161,Alibi,0.278161
4,4,JP Centre St Farmers Market,42.312173,-71.114370,10,Mass Gen. Hosp./Tea Leaves & Coffee Beans,0.100539,Mass General Hospital Cafe,0.100539,BIG DADDY HOT DOGS,...,Massachusetts Eye & Ear Infirmary Eye Opener Cafe,0.213857,RIVERSIDE CAFE-LEV1,0.261843,"YAWKEY CENTER ""COFFEE SOUTH""",0.261843,Scampo,0.275161,Alibi,0.278161
5,5,Roslindale Farmers Market,42.286545,-71.128441,10,Mass Gen. Hosp./Tea Leaves & Coffee Beans,0.100539,Mass General Hospital Cafe,0.100539,BIG DADDY HOT DOGS,...,Massachusetts Eye & Ear Infirmary Eye Opener Cafe,0.213857,RIVERSIDE CAFE-LEV1,0.261843,"YAWKEY CENTER ""COFFEE SOUTH""",0.261843,Scampo,0.275161,Alibi,0.278161
6,6,Dudley Town Common Farmers Market,42.325115,-71.074876,10,Mass Gen. Hosp./Tea Leaves & Coffee Beans,0.100539,Mass General Hospital Cafe,0.100539,BIG DADDY HOT DOGS,...,Massachusetts Eye & Ear Infirmary Eye Opener Cafe,0.213857,RIVERSIDE CAFE-LEV1,0.261843,"YAWKEY CENTER ""COFFEE SOUTH""",0.261843,Scampo,0.275161,Alibi,0.278161


In [59]:
farmers_markets_top_restaurants.to_csv('farmers_markets_top_restaurants_k_{}_d_{}.csv'.format(str(k), str(d)), index=False)

## Murals

In [68]:
data = pd.read_csv(murals_source)
data

,Artist,Location,Latitude,Longitude,Title,Year,Description,Neighborhood
0,"Wen-ti Tsen, Yuon Zuo","Oxford St, Boston, MA 02111",42.351697,-71.060571,Chinatown Heritage,1988,The Chinatown Heritage Mural is a copy of a 17...,Chinatown
1,"Vyal ""Vyal One"" Reyes","Underground at Ink Block, 90 Traveler Street, ...",42.345024,-71.060529,Untitled,2017,"Vyal ""Vyal One"" Reyes usualy includes the styl...",South End
2,"Victor ""Marka27"" Quiñonez, Rob ""ProBlak"" Gibbs","Underground at Ink Block, 90 Traveler Street, ...",42.345024,-71.060529,Don’t Let Me Be Misunderstood,2020,"In summer 2020, local artists Rob ""ProBlak"" Gi...",South End
3,"Victor ""Marka27"" Quiñonez, David ""Don Rimx"" Se...","Maurice J Tobin School, 40 Smith St, Roxbury, ...",42.333102,-71.098338,Untitled (Tobin School),2018,One of the many collaborations adorning the To...,Roxbury
4,"Victor ""Marka27"" Quiñonez","Washington Manor, 1701 Washington St. South End",42.337542,-71.076125,Souledad,2022,The City of Boston commissioned artist Victor ...,South End
...,...,...,...,...,...,...,...,...
136,Alex Cook,"Norfolk St. and Ferndale St., Dorchester, MA 0...",42.288290,-71.077099,Beholding the Holy City (Father and Daughter E...,2019,Alex Cook worked with youth from The Boston Pr...,Dorchester
137,Alex Cook,"Centre St. and Starr Lane, Jamaica Plain, MA 0...",42.313140,-71.114120,Starr Lane Landscape,2004,This brightly colored mural depicts several ch...,Jamaica Plain
138,Alex Cook,"Whittier School Apartments, 60 Southern Ave, D...",42.290010,-71.074899,Untitled,2021,Alex worked with youth from The Boston Project...,Dorchester
139,Alex Cook,"618 Shawmut Ave, Roxbury, MA 02118",42.335850,-71.080572,She Sees More Than The World Will Show Her,2009,This mural by Alex Cook features a woman stand...,Roxbury


In [69]:
meta_dists = []

for i in range(len(data)):
    market = (data["Latitude"][i], data["Longitude"][i])
    dists = []

    for j in range(len(restaurants)):
        stat = (restaurants['latitude'][j], restaurants['longitude'][j])
        distance = geopy.distance.geodesic(market, stat).km
        dists += [(distance, restaurants['businessname'][j])]

    meta_dists += [sorted(dists)[0:k]]

In [70]:
meta_dists

[[(0.019029980464256885, 'Nan Xiang Express'),
  (0.021846474916135613, 'Jibei Chuan'),
  (0.026088280187566083, 'Five Spices House'),
  (0.032950330514313904, 'Avana Market Place'),
  (0.032950330514313904, 'Friendship BBQ'),
  (0.03364183307041875, 'WTEA'),
  (0.03669668069436438, 'Dumpling King'),
  (0.03925114026178999, 'TAIWAN CAFE'),
  (0.04812469309959213, 'ASIAN GARDEN RESTAURANT INC.'),
  (0.048324674422715, 'Ten Second Noodles')],
 [(0.11793528585318265, 'AC Hotel by Marriot'),
  (0.1903486235946167, 'Capital One Cafe'),
  (0.1903486235946167, 'PLNT Burger'),
  (0.1903486235946167, 'WHOLE FOODS MARKET'),
  (0.22940040797220038, 'Bar Mezzana'),
  (0.2325655555992199, 'Fuji at Ink Block'),
  (0.2325655555992199, 'Yellow Door Taqueria'),
  (0.23297392105899614, 'Sweetgreen'),
  (0.2344675028574048, 'Cuppa Coffee'),
  (0.23529102193094503, 'Baleia')],
 [(0.11793528585318265, 'AC Hotel by Marriot'),
  (0.1903486235946167, 'Capital One Cafe'),
  (0.1903486235946167, 'PLNT Burger'),

In [71]:
murals_top_restaurants = pd.DataFrame(columns=['OBJECTID', 'Title', 'Artist', 'Avg. Lat', 'Avg. Long', 'NUM STATIONS'])


for i in range(k):
    murals_top_restaurants[("Station {}".format(str(i+1)))] = []
    murals_top_restaurants[("Station {} Dist ".format(str(i+1)))] = []

In [66]:
murals_top_restaurants

,OBJECTID,Title,Artist,Avg. Lat,Avg. Long,NUM STATIONS,Station 1,Station 1 Dist,Station 2,Station 2 Dist,...,Station 6,Station 6 Dist,Station 7,Station 7 Dist,Station 8,Station 8 Dist,Station 9,Station 9 Dist,Station 10,Station 10 Dist


In [73]:
for i in range(len(meta_dists)):
    
    title = data['Title'][i]
    artist = data['Artist'][i]
    lat = data['Latitude'][i]
    long = data['Longitude'][i]
    
    
    to_return = [i, title, artist, lat, long] 
    to_return_2 = []
    
    num = 0
    
    for j in range(k):
        stat = meta_dists[i][j][1]
        stat_d = meta_dists[i][j][0]
        
        if stat_d < d:
            num += 1
        
        to_return_2 += [stat, stat_d]
        
    
    to_return_2 = [num] + to_return_2

    to_return += to_return_2
    
    murals_top_restaurants.loc[len(murals_top_restaurants.index)] = to_return


In [74]:
murals_top_restaurants

,OBJECTID,Title,Artist,Avg. Lat,Avg. Long,NUM STATIONS,Station 1,Station 1 Dist,Station 2,Station 2 Dist,...,Station 6,Station 6 Dist,Station 7,Station 7 Dist,Station 8,Station 8 Dist,Station 9,Station 9 Dist,Station 10,Station 10 Dist
0,0,Chinatown Heritage,"Wen-ti Tsen, Yuon Zuo",42.351697,-71.060571,10,Nan Xiang Express,0.019030,Jibei Chuan,0.021846,...,WTEA,0.033642,Dumpling King,0.036697,TAIWAN CAFE,0.039251,ASIAN GARDEN RESTAURANT INC.,0.048125,Ten Second Noodles,0.048325
1,1,Untitled,"Vyal ""Vyal One"" Reyes",42.345024,-71.060529,10,AC Hotel by Marriot,0.117935,Capital One Cafe,0.190349,...,Fuji at Ink Block,0.232566,Yellow Door Taqueria,0.232566,Sweetgreen,0.232974,Cuppa Coffee,0.234468,Baleia,0.235291
2,2,Don’t Let Me Be Misunderstood,"Victor ""Marka27"" Quiñonez, Rob ""ProBlak"" Gibbs",42.345024,-71.060529,10,AC Hotel by Marriot,0.117935,Capital One Cafe,0.190349,...,Fuji at Ink Block,0.232566,Yellow Door Taqueria,0.232566,Sweetgreen,0.232974,Cuppa Coffee,0.234468,Baleia,0.235291
3,3,Untitled (Tobin School),"Victor ""Marka27"" Quiñonez, David ""Don Rimx"" Se...",42.333102,-71.098338,10,CHACHO'S PIZZA,0.137275,Milkweed,0.143835,...,MIKES DONUTS,0.162526,Lilly Gourmet Pasta Express,0.167041,Nachlo,0.185081,A K's Take Out & Delivery,0.215456,Cilantro Latin Kitchen,0.251595
4,4,Souledad,"Victor ""Marka27"" Quiñonez",42.337542,-71.076125,10,Yunnan Kitchen,0.066937,TORO,0.070256,...,El Centro,0.115993,Code 10 Restaurant Cafe,0.124669,FLOUR BAKERY & CAFE,0.134188,ORINOCO:A LATIN KITCHEN,0.151794,Anoush Ella,0.237515
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,136,Beholding the Holy City (Father and Daughter E...,Alex Cook,42.288290,-71.077099,2,Stalex Pizza,0.080832,ONLY ONE JAMAICAN RESTAURANT,0.177638,...,ORIENTAL HOUSE,0.622931,La La Restaurant,0.687723,BILLY'S TEXAS B.B.Q.,0.689566,China Station Restaurant,0.689566,Essential Herb & Juice Cafe,0.765181
137,137,Starr Lane Landscape,Alex Cook,42.313140,-71.114120,10,J.P. LICKS INC.,0.039657,Chilacates,0.048178,...,Shea Butter Smoothies,0.135060,Sweet Rice,0.153811,Galway House,0.174616,The Joint,0.182605,Flavor Boom,0.183819
138,138,Untitled,Alex Cook,42.290010,-71.074899,2,Taste of Eden,0.191680,McDonald's,0.253303,...,BILLY'S TEXAS B.B.Q.,0.426456,China Station Restaurant,0.426456,ONLY ONE JAMAICAN RESTAURANT,0.441143,Essential Herb & Juice Cafe,0.509135,Raphael's Restaurant,0.560891
139,139,She Sees More Than The World Will Show Her,Alex Cook,42.335850,-71.080572,3,Pizzatalia,0.054534,Residence Inn,0.269223,...,MIDA Restaurant,0.348744,Yunnan Kitchen,0.353011,Subway,0.355029,Jugos,0.383350,Ilona,0.387753


In [76]:
murals_top_restaurants.to_csv('murals_top_restaurants_k_{}_d_{}.csv'.format(str(k), str(d)), index=False)